## Libraries

In [1]:
from datetime import datetime
from scipy.stats import norm
import numpy as np
import pandas as pd
import requests
from enum import Enum

class ActionType(Enum):
    BUY = 1
    DONOTHING = 0
    SELL = -1

#### Function to make an API call to Binance

In [2]:
def make_api_call(base_url, endpoint="", method="GET", **kwargs):
    # Construct the full URL
    full_url = f'{base_url}{endpoint}'

    # Make the API call
    response = requests.request(method=method, url=full_url, **kwargs)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        return response
    else:
        # If the request was not successful, raise an exception with the error message
        raise Exception(f'API request failed with status code {response.status_code}: {response.text}')

## Getting data

In [3]:
def get_binance_historical_data(symbol, interval, start_date, end_date):
    
    # define basic parameters for call
    base_url = 'https://fapi.binance.com'
    endpoint = '/fapi/v1/klines'
    method = 'GET'
    
    # Set the start time parameter in the params dictionary
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': 1500,
        'startTime': start_date, # Start time in milliseconds
        'endTime': end_date # end time in milliseconds
    }


    # Make initial API call to get candles
    response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)
    
    # initalize candles data
    candles_data = []

    # Append the received candles to the list
    candles_data.extend(response.json())

    # Update the start time for the next API call
    params['startTime'] = candles_data[-1][0] + 1 # last candle open_time + 1ms    

    while len(response.json()) > 0:
        # Make the next API call
        response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)

        # Append the received candles to the list
        candles_data.extend(response.json())

        # Update the start time for the next API call
        params['startTime'] = candles_data[-1][0] + 1 # last candle open_time + 1ms
        
        if params['startTime'] > params['endTime']:
            break

            

    
    # Wrap the candles data as a pandas DataFrame
    columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
               'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    dtype={
    'open_time': 'datetime64[ms, Asia/Jerusalem]',
    'open': 'float64',
    'high': 'float64',
    'low': 'float64',
    'close': 'float64',
    'volume': 'float64',
    'close_time': 'datetime64[ms, Asia/Jerusalem]',
    'quote_asset_volume': 'float64',
    'number_of_trades': 'int64',
    'taker_buy_base_asset_volume': 'float64',
    'taker_buy_quote_asset_volume': 'float64',
    'ignore': 'float64'
    }
    
    df = pd.DataFrame(candles_data, columns=columns)
    df = df.astype(dtype)

    return df

In [4]:
symbol = 'BTCUSDT'
interval = '30m'
start_date = int(datetime(year=2023, month=1, day=1).timestamp() * 1000)
end_date = int(datetime(year=2024, month=1, day=1).timestamp() * 1000)

btcusdt_df = get_binance_historical_data(symbol, '30m', start_date, end_date)
btcusdt_df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,2023-01-01 00:29:59.999000+02:00,1.279086e+08,35451,3338.726,5.520766e+07,0.0
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,2023-01-01 00:59:59.999000+02:00,1.473757e+08,41907,3740.250,6.174233e+07,0.0
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,2023-01-01 01:29:59.999000+02:00,8.608995e+07,27411,2761.688,4.558437e+07,0.0
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,2023-01-01 01:59:59.999000+02:00,5.171514e+07,17078,1772.384,2.930818e+07,0.0
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,2023-01-01 02:29:59.999000+02:00,4.681919e+07,16561,1373.220,2.269722e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17516,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,2023-12-31 22:29:59.999000+02:00,1.091479e+08,28539,1367.677,5.833755e+07,0.0
17517,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,2023-12-31 22:59:59.999000+02:00,9.251599e+07,27578,1098.223,4.678832e+07,0.0
17518,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,2023-12-31 23:29:59.999000+02:00,6.265517e+07,20273,862.639,3.679818e+07,0.0
17519,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,2023-12-31 23:59:59.999000+02:00,9.909701e+07,24200,1099.718,4.687127e+07,0.0


## **UTBot**
### **The Logic Behind it:**
The UTBot indicator is more carefull in it's signals overall the higher the key value is.
* The key value is setting the overall size of the threshold, the higher the key value the greater the threshold is, and accordingly it is harder for the trailing stop to change.

    that is because when the threshold is bigger, the trailing stop doesnt change unless the current close is greater than the previous trailing stop and the previous close is smaller than the previous trailing stop, or the opposite.

    when both the current and the previous close are greater than the previous trailing stop or smaller than it the trailing stop can only will change only if the difference between the current close and the previous close is greater than the current threshold.

    and when the key value is greater the threshold becomes greater aswell.

The UTBot indicator is more carefull in it's signals over the passing of time the longer the ATR Period is set.
* The ATR Period is setting the weakening of the effect each TR value has on the threshold, meaning the more the TR values "agree" with each other, the more representetive the ATR value of the TRs' values.

    from this comes to play how does this value affect the threshold, the more there are TR values that are greater, the higher the threshold, and the same applys for when the there are more TR values that are smaller.

    over time the threshold changes less speratically the greater the ATR Period because there are more TR values to keep the ATR and the threshold values similar to their previous values.

    in other words the threshold is more stable over time when the longer the ATR Period is.

in the video there are 2 UTBot indictors used, both have the same key value, but the indicator that gives the Buy signal is more carefull than the indicator that gives the Sell signal when it "decides" to give the signal.

this means that to give a Buy signal the indicators wait for longer than for giving a Sell signal, once there is some slight loss you should Sell but when there is there some gain you should wait and see if this gain is consistent enough.

### **How it works:**
1. Calculate the $TR_t$ for every sample $t$ in the data.

    **TR Formula:** $\;\:TR_t = \max\{High_t,\;Close_{t-1}\} - \min\{Low_t,\;Close_{t-1}\}$

2. Calculate using the $TR_t$ the $ATR_t$ for every sample $t$ in the data relative to the set $ATR\,\:Period$ ($N$ in the formula).

    **ATR Formula:** $\;\:ATR_t = \frac{\sum_{i=0}^{N-1} TR_{t-i}}{N}$

3. Calculate using the $ATR_t$ the $LossThreshold_t$ for every sample $t$ in the data relative to the set $KeyVal$.

    **Loss Threshold Formula:** $\;\:LossThreshold_t = KeyVal \times ATR_t$

4. Set $TrailingStop_{\,0} = 0$

5. Find the $TrailingStop_{t}$ of for every sample $t$ in the data (not including $0$) in accordance to the following rules:
    * When $Close_t>TrailingStop_{t-1}\;$ **&** $\;Close_{t-1}>TrailingStop_{t-1}$:
    $$TrailingStop_t = \max\{TrailingStop_{t-1},\;Close_t - LossThreshold_t\}$$
    * When $Close_t<TrailingStop_{t-1}\;$ **&** $\;Close_{t-1}<TrailingStop_{t-1}$:
    $$TrailingStop_t = \min\{TrailingStop_{t-1},\;Close_t + LossThreshold_t\}$$
    * When $Close_t>TrailingStop_{t-1}\;$ **&** $\;Close_{t-1}<TrailingStop_{t-1}$:
    $$TrailingStop_t = close_t - LossThreshold_t$$
    * In any other case:
    $$TrailingStop_t = close_t + LossThreshold_t$$

6. Set the signals in accordance to the following rules:
    * When $close_t>TrailingStop_t\;$ **&** $\;Close_{t-1}<TrailingStop_{t-1}$ set $Signal_t = Buy$
    * When $close_t<TrailingStop_t\;$ **&** $\;Close_{t-1}>TrailingStop_{t-1}$ set $Signal_t = Sell$
    * In any other case set $Signal_t = DoNothing$

#### Helping Functions

In [ ]:
def TR(close: pd.Series, high: pd.Series, low: pd.Series) -> pd.Series:
    return (high.where(high > close.shift(1), close.shift(1)) - low.where(low < close.shift(1), close.shift(1)))

def ATR(TRsrs: pd.Series, atr_length: int) -> pd.Series:
    return TRsrs.rolling(window = atr_length).mean()

def crossOver(a: pd.Series, b: pd.Series) -> pd.Series:
    return (a > b) & (a.shift(1) < b.shift(1))

##### Main Function

Because the presentor used 2 UTBot indicators we decided to make the UTBot implementation a little more customizable

Here the default settings are the same as the presentor has set in the video

In [92]:
def UTBot(close, high, low, buyKeyVal = 2, buyATRlen = 300, sellKeyVal = 2, sellATRlen = 1) -> pd.Series:
    # Steps 1, 2 and 3
    lossThrshBuy = buyKeyVal * ATR(TR(close, high, low), buyATRlen)
    lossThrshSell = sellKeyVal * ATR(TR(close, high, low), sellATRlen)
    trailStopBuy = 0 * lossThrshBuy
    trailStopSell = 0 * lossThrshSell

    # Step 4
    for i in range(1, len(trailStopBuy)):
        # the previous buy trailing stop is lower than the current and previous close
        if (close[i] > trailStopBuy[i - 1]) & (close[i - 1] > trailStopBuy[i - 1]):
            trailStopBuy[i] = max(trailStopBuy[i - 1], close[i] - lossThrshBuy[i])
        # the previous buy trailing stop is higher than the current and previous close
        elif (close[i] < trailStopBuy[i - 1]) & (close[i - 1] < trailStopBuy[i - 1]):
            trailStopBuy[i] = min(trailStopBuy[i - 1], close[i] + lossThrshBuy[i])
        # the previous buy trailing stop is lower than the current close and higher than the previous close
        elif (close[i] > trailStopBuy[i - 1]):
            trailStopBuy[i] = close[i] - lossThrshBuy[i]
        # the previous buy trailing stop is higher than the current close and lower than the previous close
        else:
            trailStopBuy[i] = close[i] + lossThrshBuy[i]
        
        # the same for the sell trailing stop
        if (close[i] > trailStopSell[i - 1]) & (close[i - 1] > trailStopSell[i - 1]):
            trailStopSell[i] = max(trailStopSell[i - 1], close[i] - lossThrshSell[i])
        elif (close[i] < trailStopSell[i - 1]) & (close[i - 1] < trailStopSell[i - 1]):
            trailStopSell[i] = min(trailStopSell[i - 1], close[i] - lossThrshSell[i])
        elif (close[i] > trailStopSell[i - 1]):
            trailStopSell[i] = close[i] - lossThrshSell[i]
        else:
            trailStopSell[i] = close[i] + lossThrshSell[i]

    # Step 5
    above = crossOver(close, trailStopBuy)
    below = crossOver(trailStopSell, close)

    # Step 6
    Action = np.select([above, below], [ActionType.BUY, ActionType.SELL], default = ActionType.DONOTHING)
    
    return Action

## STC Osilator
### The Logic Behind it:
The STC Osilator takes into acount the exponential changes of 2 Periods that end at the same time.

These exponential changes are expressed within the EMA 
### How it works:
1. Calculate the $FastEMA_t$ and the $SlowEMA_t$ of the $Close$ value for every sample $t$ in the data relaitve to the set $Fast\:Length$ and $Slow\:Length$, both of the EMA's are calculated as shown in class where the n is set as their apropriate $^{\{Type\}}Length$.

    Here we used the ***ewm*** method combined with ***mean***, as shown in class.

2. Calculate the $MacdDiff_t$ for every sample $t$ in the data.

    **MacdDiff Formula:** $\; MacdDiff_t = FastEMA_t - SlowEMA_t$

3. Normalize and Smooth (twice) the resulting line that is created when ploting the $MacdDiff_t$ values using the set $SmoothFactor$ and $STCLen$.

    ##### The Smoothing Process of Variable $Val$
    1. Set $SmoothVal_t$ as a duplicate of $Val_t$ for every $t$ in the data.
    2. Set $SmoothVal_t=Val_t$ when $SmoothVal_{t-1}$ is $Na$
        
        when $SmoothVal_{t-1}$ is not $Na$ set $SmoothVal_t = SmoothVal_{t-1} + SmoothFactor \times (Val_t - SmoothVal_{t-1})$

    ##### The Normalized Smoothing Process of Variable $Val$
    1. Set $RangeLow_t = \min\{Val_{*t}\}\,$, $\;RangeHigh_t = \min\{Val_{*t}\}$ and $Diff_t = RangeHigh_t - RangeLow_t$
    
        where $Val_{*t} = \{Val_k\:|\:k \in [t-STCLen,\;t]\}$
    2. Set $NormVal_t$ as a duplicate of $Val_t$ for every $t$ in the data.
    3. Where $Diff_t > 0$ (should be all of them, but in case of errors) set $NormVal_t = \frac{Val_t - RangeLow_t}{Diff_t \times 100}$ and fill apropriatly the values where there happen to be an error.
    4. Smooth $NormVal$ relative to the set $SmoothFactor$.
    5. add the 2 final steps

#### Helping Functions

In [ ]:
def SmoothSrs(srs, smoothFact):
    smoothed_srs = srs.copy()
    for i in range(1, len(smoothed_srs)):
        if np.isnan(smoothed_srs[i-1]):
            smoothed_srs[i] = srs[i]
        else:
            smoothed_srs[i] = smoothed_srs[i-1] + smoothFact * (srs[i] - smoothed_srs[i-1])
    return smoothed_srs

def normNsmooth(srs, stc_length, smoothFact):
    # finding the lowest and highest range
    lowest = srs.rolling(stc_length).min()
    highestRange = srs.rolling(stc_length).max() - lowest
    
    # normalizing srs
    normalizedsrs = srs.copy()
    normalizedsrs[highestRange > 0] = ((srs - lowest) / highestRange * 100)*(highestRange > 0)
    normalizedsrs[highestRange <= 0] = np.nan
    normalizedsrs.ffill(inplace = True)
    
    # smoothing the srs
    return SmoothSrs(normalizedsrs, smoothFact)

##### Main Function
Default settings are the same as done in the video

In [ ]:
def stcOsilator(srs, fast_length = 27, slow_length = 50, stc_length = 80, smoothFact = 0.5):
    # ema calculation for fast and slow length's
    fast_ema = srs.ewm(span = fast_length).mean()
    slow_ema = srs.ewm(span = slow_length).mean()

    # MacdDiff calculation and smoothing
    MacdDiff = fast_ema - slow_ema
    smoothedMacd = normNsmooth(MacdDiff, stc_length, smoothFact)
    FinalSTC = normNsmooth(smoothedMacd, stc_length, smoothFact)
    
    thirdQuar = pd.Series(norm.ppf(0.75), index = FinalSTC.index)

    above = crossOver(FinalSTC, thirdQuar)
    descent = (FinalSTC < FinalSTC.shift(1)).ffill(inplace = False)
    below = crossOver(-1 * thirdQuar, FinalSTC)
    rise = (FinalSTC > FinalSTC.shift(1)).ffill(inplace = False)

    Action = np.select([above & descent, below & rise], [ActionType.SELL, ActionType.BUY], default = ActionType.DONOTHING)

    return Action

In [93]:
df['UTBot_indicator'] = UTBot(close = df.close, high = df.high, low = df.low)
df['STC_indicator'] = stcOsilator(srs = df.close)
df

C:\Users\tomge\AppData\Local\Temp\ipykernel_26344\283304217.py:80: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normalizedsrs.fillna(method = 'ffill', inplace = True)
C:\Users\tomge\AppData\Local\Temp\ipykernel_26344\283304217.py:80: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normalizedsrs.fillna(method = 'ffill', inplace = True)


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,UTBot_indicator,STC_indicator
0,2024-01-01 00:00:00+02:00,42559.0,42613.0,42466.1,42608.9,2926.528,2024-01-01 00:29:59.999000+02:00,1.245095e+08,31338,1320.000,5.617103e+07,0.0,ActionType.DONOTHING,0.000000
1,2024-01-01 00:30:00+02:00,42608.9,42629.5,42111.9,42294.8,9025.818,2024-01-01 00:59:59.999000+02:00,3.821358e+08,89174,3701.256,1.566922e+08,0.0,ActionType.DONOTHING,-0.921815
2,2024-01-01 01:00:00+02:00,42294.8,42380.1,42083.1,42211.2,8760.876,2024-01-01 01:29:59.999000+02:00,3.699976e+08,82898,3594.587,1.518230e+08,0.0,ActionType.BUY,-2.457324
3,2024-01-01 01:30:00+02:00,42211.3,42315.6,42180.8,42314.0,2915.589,2024-01-01 01:59:59.999000+02:00,1.232041e+08,33344,1624.211,6.863622e+07,0.0,ActionType.DONOTHING,-3.566754
4,2024-01-01 02:00:00+02:00,42314.0,42603.2,42289.6,42458.5,5940.016,2024-01-01 02:29:59.999000+02:00,2.522419e+08,58673,3456.112,1.467360e+08,0.0,ActionType.DONOTHING,-3.447998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2024-01-06 22:00:00+02:00,43903.6,43960.7,43645.0,43855.7,5595.282,2024-01-06 22:29:59.999000+02:00,2.450022e+08,62110,2374.617,1.040008e+08,0.0,ActionType.DONOTHING,96.573092
285,2024-01-06 22:30:00+02:00,43855.7,43897.4,43775.3,43816.4,1752.555,2024-01-06 22:59:59.999000+02:00,7.683397e+07,26867,707.315,3.100777e+07,0.0,ActionType.DONOTHING,49.120926
286,2024-01-06 23:00:00+02:00,43816.3,43901.7,43712.1,43868.0,2106.253,2024-01-06 23:29:59.999000+02:00,9.226863e+07,32340,1048.501,4.593668e+07,0.0,ActionType.DONOTHING,24.560463
287,2024-01-06 23:30:00+02:00,43868.0,43930.4,43798.1,43881.3,2436.663,2024-01-06 23:59:59.999000+02:00,1.069033e+08,34767,1351.523,5.930249e+07,0.0,ActionType.DONOTHING,12.280232
